In [1]:
import os
from openpyxl import load_workbook, Workbook
from openpyxl.cell.cell import MergedCell
import re

# Define paths to the source workbooks
source_corn_workbook_path = r'C:\Users\benja\Documents\Corn_Soy_Dashboard\Data\Trials\2023-Corn-Grain-Tables.xlsx'
source_soy_workbook_path = r'C:\Users\benja\Documents\Corn_Soy_Dashboard\Data\Trials\2023-Soybean-Tables.xlsx'
corn_trials_folder = 'CornTrials'
soy_trials_folder = 'SoyTrials'

# Ensure folders exist
os.makedirs(corn_trials_folder, exist_ok=True)
os.makedirs(soy_trials_folder, exist_ok=True)

# List of locations for corn and soy trials
corn_locations = [
    "Greeneville", "Jackson", "Knoxville", "Memphis", "Milan", "Spring Hill", "Springfield",
    "Bradley", "Carroll", "Crockett", "Decatur", "Dyer", "Fayette", "Franklin", "Gibson", "Giles",
    "Hardeman", "Haywood", "HenryB", "HenryT", "Hickman", "Jefferson", "Lauderdale", "Loudon",
    "Macon", "Madison", "Maury", "Meigs", "Montgomery", "Obion", "Tipton", "Warren", "Weakley"
]

soy_locations = [
    "Greeneville", "Jackson", "Knoxville", "Memphis", "Milan", "Spring Hill", "Springfield",
    "Crockett", "Decatur", "Dyer", "Fayette", "Franklin", "Gibson", "Giles", "Hamblen", "Hardeman",
    "Hardin", "Hawkins", "Haywood", "Henderson", "Henry", "Jefferson", "Lake", "Lauderdale",
    "Macon", "Madison", "Maury", "Perry", "Weakley"
]

# Function to copy a sheet's contents manually, skipping styles that cause issues
def copy_sheet_contents(source_sheet, target_sheet):
    for row in source_sheet.iter_rows():
        for cell in row:
            if isinstance(cell, MergedCell):
                continue  # Skip merged cells, they are handled separately

            new_cell = target_sheet.cell(row=cell.row, column=cell.col_idx, value=cell.value)
            if cell.has_style:
                try:
                    new_cell.font = cell.font
                    new_cell.border = cell.border
                    new_cell.fill = cell.fill
                    new_cell.number_format = cell.number_format
                    new_cell.protection = cell.protection
                    new_cell.alignment = cell.alignment
                except TypeError:
                    pass  # Skip styles that cause errors

    # Copy merged cells
    for merged_cell_range in source_sheet.merged_cells.ranges:
        target_sheet.merge_cells(str(merged_cell_range))
        start_cell = source_sheet[str(merged_cell_range).split(":")[0]]
        target_cell = target_sheet[str(merged_cell_range).split(":")[0]]
        target_cell.value = start_cell.value
        if start_cell.has_style:
            try:
                target_cell.font = start_cell.font
                target_cell.border = start_cell.border
                target_cell.fill = start_cell.fill
                target_cell.number_format = start_cell.number_format
                target_cell.protection = start_cell.protection
                target_cell.alignment = start_cell.alignment
            except TypeError:
                pass  # Skip styles that cause errors

    # Copy column widths and row heights
    for col in source_sheet.column_dimensions:
        target_sheet.column_dimensions[col].width = source_sheet.column_dimensions[col].width
    for row in source_sheet.row_dimensions:
        target_sheet.row_dimensions[row].height = source_sheet.row_dimensions[row].height

# Function to sanitize filenames (remove invalid characters)
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "", filename)

# Function to check if a location is present in the sheet's content (case-insensitive)
def location_in_sheet_content(sheet, location):
    location_lower = location.lower()
    for row in sheet.iter_rows(values_only=True):
        for cell in row:
            if cell and location_lower in str(cell).lower():
                return True
    return False

# Function to create workbooks for each location by searching within sheet contents
def create_location_workbook(source_path, output_folder, locations, crop_type):
    source_wb = load_workbook(source_path, data_only=True)

    for location in locations:
        new_wb = Workbook()
        new_wb.remove(new_wb.active)  # Remove default sheet

        sheets_added = False
        for sheet_name in source_wb.sheetnames:
            if source_wb[sheet_name].sheet_state == 'hidden':
                continue  # Skip hidden sheets
            
            source_sheet = source_wb[sheet_name]
            if location_in_sheet_content(source_sheet, location):
                new_sheet = new_wb.create_sheet(title=source_sheet.title)
                copy_sheet_contents(source_sheet, new_sheet)
                sheets_added = True

        if sheets_added:
            output_filename = sanitize_filename(f'{crop_type}_{location}.xlsx')
            output_path = os.path.join(output_folder, output_filename)
            new_wb.save(output_path)
            print(f"Created workbook: {output_path}")
        else:
            print(f"No sheets found for {location} in {crop_type} workbook.")

# Process Corn and Soy workbooks
create_location_workbook(source_corn_workbook_path, corn_trials_folder, corn_locations, 'Corn')
create_location_workbook(source_soy_workbook_path, soy_trials_folder, soy_locations, 'Soy')

Created workbook: CornTrials\Corn_Greeneville.xlsx
Created workbook: CornTrials\Corn_Jackson.xlsx
Created workbook: CornTrials\Corn_Knoxville.xlsx
Created workbook: CornTrials\Corn_Memphis.xlsx
Created workbook: CornTrials\Corn_Milan.xlsx
Created workbook: CornTrials\Corn_Spring Hill.xlsx
Created workbook: CornTrials\Corn_Springfield.xlsx
Created workbook: CornTrials\Corn_Bradley.xlsx
Created workbook: CornTrials\Corn_Carroll.xlsx
Created workbook: CornTrials\Corn_Crockett.xlsx
Created workbook: CornTrials\Corn_Decatur.xlsx
Created workbook: CornTrials\Corn_Dyer.xlsx
Created workbook: CornTrials\Corn_Fayette.xlsx
Created workbook: CornTrials\Corn_Franklin.xlsx
Created workbook: CornTrials\Corn_Gibson.xlsx
Created workbook: CornTrials\Corn_Giles.xlsx
Created workbook: CornTrials\Corn_Hardeman.xlsx
Created workbook: CornTrials\Corn_Haywood.xlsx
Created workbook: CornTrials\Corn_HenryB.xlsx
Created workbook: CornTrials\Corn_HenryT.xlsx
Created workbook: CornTrials\Corn_Hickman.xlsx
Create